In [ ]:
from tictacchess import *


scores={'Vitoria': 3, 'Empate': 1}

def traduzPontos(tabela):
    tabelaScore={}
    empates=tabela['Empate']
    for x in tabela:
        if x != 'Empate':
            tabelaScore[x]=scores['Vitoria']*tabela[x]+scores['Empate']*empates
    return tabelaScore

def jogaNpares(jogo,n,jog1,jog2):
    name_jog1=jog1.__name__
    name_jog2=jog2.__name__
    tabelaPrim={name_jog1:0, name_jog2:0, 'Empate':0}
    tabelaSeg={name_jog1:0, name_jog2:0, 'Empate':0}
    tabela={}
    for i in range(n):
        #_,_,vencedor=jogo.jogar(jog1,jog2,verbose=False)
        vencedor=jogo.jogar(jog1,jog2,verbose=False)

        if vencedor>0:
            vencedor=name_jog1
        elif vencedor<0:
            print(str(i) + ":PRETAS")
            vencedor=name_jog2
        else:
            vencedor='Empate'
        tabelaPrim[vencedor]+=1
        vencedor=jogo.jogar(jog2,jog1,verbose=False)
        if vencedor>0:
            print(str(i) + ":BRANCAS")
            vencedor=name_jog2
        elif vencedor<0:
            vencedor=name_jog1
        else:
            vencedor='Empate'
        tabelaSeg[vencedor]+=1
    for x in tabelaPrim:
        tabela[x]=tabelaPrim[x]+tabelaSeg[x]
    return tabelaPrim,tabelaSeg,tabela,traduzPontos(tabela)

def func_pecas(estado,jogador) :
    clone=copy.deepcopy(estado)
    n_pecas_jogador = len(clone.player_used_pieces(clone.to_move))
    n_pecas_adversario = len(clone.player_used_pieces(clone.other()))
    return n_pecas_jogador - n_pecas_adversario

def func_tactic(estado,jogador) :
    clone=copy.deepcopy(estado)
    winner = clone.have_winner()
    if winner != None:
        return infinity if winner==jogador else -infinity
    # se não reconhece o final do jogo, verifica quem tem três em linha:
    almost_winner = clone.n_in_row(3)
    if almost_winner == None or almost_winner == 'BOTH':
        return 0
    return 1 if almost_winner==jogador else -1


def func_tactic_e_pecas(estado,jogador):
    return func_tactic(estado,jogador) + func_pecas(estado,jogador)

def jogador_tactic_e_pecas_3(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo,3,eval_fn=func_tactic_e_pecas)

def incorpora(tabela,tx):
    for jog in tx:
        if jog not in tabela:
            tabela[jog]=tx[jog]
        else:
            tabela[jog]+=tx[jog]

def torneio(n,jogadores):
    jogo=TicTacChess()
    tabela={}
    for i in range(len(jogadores)-1):
        jog1=jogadores[i]
        for j in range(i+1,len(jogadores)):
            jog2=jogadores[j]
            print(jog1.__name__,'vs',jog2.__name__)
            _,_,_,tabelaX = jogaNpares(jogo,n,jog1,jog2)
            incorpora(tabela,tabelaX)
    #return tabela
    print(dict(sorted(tabela.items(), key=lambda x: x[1],reverse=True)))


In [6]:
def check_n_in_row(board, n):
    upper_count = lower_count = 0

    # Create a 4x4 grid initialized with empty strings
    grid = [['' for _ in range(4)] for _ in range(4)]

    # Fill the grid with the pieces from the board dictionary
    for piece, (row, col) in board.items():
        grid[row][col] = piece

    # Check rows and columns
    for i in range(4):
        for j in range(4 - n + 1):
            if all(grid[i][j + k].isupper() for k in range(n)):
                upper_count += 1
            if all(grid[i][j + k].islower() for k in range(n)):
                lower_count += 1
            if all(grid[j + k][i].isupper() for k in range(n)):
                upper_count += 1
            if all(grid[j + k][i].islower() for k in range(n)):
                lower_count += 1

    # Check diagonals
    for i in range(4 - n + 1):
        for j in range(4 - n + 1):
            if all(grid[i + k][j + k].isupper() for k in range(n)):
                upper_count += 1
            if all(grid[i + k][j + k].islower() for k in range(n)):
                lower_count += 1
            if all(grid[i + k][j + n - 1 - k].isupper() for k in range(n)):
                upper_count += 1
            if all(grid[i + k][j + n - 1 - k].islower() for k in range(n)):
                lower_count += 1

    if upper_count > 0 and lower_count > 0:
        return 'BOTH'
    elif upper_count > 0:
        return 'WHITE'
    elif lower_count > 0:
        return 'BLACK'
    else:
        return None


def check_in_row(piece_cells):
    if len(piece_cells) < 2:
        return False
    piece_cells = sorted(piece_cells)
    for i in range(len(piece_cells) - 1):
        if piece_cells[i][0] == piece_cells[i + 1][0] and piece_cells[i][1] == piece_cells[i + 1][1] - 1:
            continue
        elif piece_cells[i][1] == piece_cells[i + 1][1] and piece_cells[i][0] == piece_cells[i + 1][0] - 1:
            continue
        elif piece_cells[i][0] == piece_cells[i + 1][0] - 1 and piece_cells[i][1] == piece_cells[i + 1][1] - 1:
            continue
        elif piece_cells[i][0] == piece_cells[i + 1][0] - 1 and piece_cells[i][1] == piece_cells[i + 1][1] + 1:
            continue
        else:
            return False
    return True

In [ ]:
prof = 3


def jogador_tactic_247(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo, prof,eval_fn=func_247)

def jogador_tactic_3(jogo,estado) :
    return alphabeta_cutoff_search_new(estado,jogo, prof,eval_fn=func_tactic)


def func_247(state, player):
    clone=copy.deepcopy(state)
    return func_winner(clone, player) + func_center(clone, player) + func_mobility(clone,player) + func_pawn_early_game(clone, player) + flexible_alignment_bonus(clone, player)

def other_player(player):
    return 'WHITE' if player == 'BLACK' else 'BLACK'

def func_pawn_early_game(state, player):
    player_pieces = state.player_pieces(player)
    if state.n_jogadas < 8 and player_pieces in ['p', 'P']:
        return -200
    return 0

def func_mobility(state, player):
    player_moves = 0
    opponent_moves = 0

    # Calculate player mobility
    player_cells, player_pieces = state.player_used_cells(player)
    for piece, _ in zip(player_pieces, player_cells):
        moves = state.possible_moves(piece)
        player_moves += len(moves) * 2

    player_cells, player_pieces = state.player_used_cells(other_player(player))
    for piece, _ in zip(player_pieces, player_cells):
        moves = state.possible_moves(piece)
        opponent_moves += len(moves) * 2

    return player_moves - opponent_moves

def flexible_alignment_bonus(state, player):
    player_cells, _ = state.player_used_cells(player)
    alignments = 0

    # Contar possíveis alinhamentos a partir de diferentes combinações
    for group in combinations(player_cells, 3):
        if in_row(sorted(group)):
            alignments += 1

    return 200 * alignments  # Recompensa maior para mais ameaças simultâneas

def is_threatening(state, player, move):
    # Pieces currently in play for the given player
    pieces = state.player_used_pieces(state.to_move)
    empty_cells = state.empty_cells(state)

    # Early game: if the player can place a new piece at the move location
    if len(pieces) < 4 and move in empty_cells:
        return 1000 if state.to_move == player else -1000

    # Check if the move aligns with the possible moves of existing pieces
    for piece in pieces:
        if move in state.possible_moves(piece):
            return 500 if state.to_move == player else -500

    # No immediate threat found
    return 0

def func_center(state, player):
    """Player places his pieces in the center"""
    player_cells, _ = state.player_used_cells(player)
    opponent_cells, _ = state.player_used_cells(other_player(player))
    center = [(1,1),(1,2),(2,1),(2,2)]

    player_control = sum(1 for cell in player_cells if cell in center)
    opponent_control = sum(1 for cell in opponent_cells if cell in center)
    return (player_control - opponent_control)

def func_winner(state, player):
    winner = state.have_winner()
    if winner != None:
        return infinity if winner == player else -infinity
    score = 0
    # se não reconhece o final do jogo, verifica quem tem três em linha:
    almost_winner = state.n_in_row(3)
    if almost_winner == player:
        # Check if current state is blocking the row of the player
        missing_cell = row_blocked(state, 3, other_player(player), player)
        threat_score  = is_threatening(state, player, missing_cell)
        score = threat_score if threat_score != 0 else 1000
    elif almost_winner == other_player(player):
        # Check if current state is blocking the row of the other player
        missing_cell = row_blocked(state, 3, player, other_player(player))
        threat_score = is_threatening(state, player, missing_cell)
        score = threat_score if threat_score != 0 else -1000

    elif almost_winner == "BOTH":
        # Check if current state is blocking the row of the player
        opponent_missing  = row_blocked(state, 3, other_player(player), player)
        score += is_threatening(state, player, opponent_missing)
        if score == 0:
            # Check if current state is blocking the row of the other player
            missing_cell = row_blocked(state, 3, player, other_player(player))
            threat_score = is_threatening(state, player, missing_cell)
            score += threat_score if threat_score != 0 else -1000

    return score - state.n_jogadas

def row_blocked(state, n, player, other):
    "Return the player or players who have n-in-row (maybe Both), or None."
    #(play,board,jogadas,capturas,pawn_direction,piece)=self
    cells1,_ = state.player_used_cells(player)
    cells2,_ = state.player_used_cells(other)
    empty_cells = state.empty_cells(state)

    if len(cells2) >= n: # we check if a group of them is in_row, assume
        for group_cells2 in combinations(cells2, n):
            group_cells2 = list(group_cells2)
            for cell1 in cells1 + empty_cells:
                group_cells2.append(cell1)
                group_cells2 = sorted(group_cells2)
                if in_row(group_cells2):
                    return cell1
                group_cells2.remove(cell1)
    return None



#brd= {'B' : (0,0), 'T' : (2,0), 'P' : (3,0),'C' : (2,2),'p' : (0,2), 't' : (1,2), 'c' : (3,3)}
#state = EstadoTicTacChess(to_move="BLACK",board=brd,n_jogadas=10,n_capturas=[0,0],pawn_direction=[1,1],last_piece="p")
#print(func_winner(state, "BLACK"))
#print(check_n_in_row(brd, 3))

jogo=TicTacChess()
for _ in range(30):
    torneio(1,[jogador_tactic_3, jogador_tactic_247])

jogador_tactic_3 vs jogador_tactic_247
   Jogadas feitas: 0
   Tabuleiro atual:
   _________
0 | . . . . | 
1 | . . . . | 
2 | . . . . | 
3 | . . . . | 
  |_________|
    0 1 2 3 
   Próximo jogador: WHITE
      Stock de peças:    ['B', 'P', 'C', 'T']    (black: ['t', 'b', 'p', 'c'])
      Capturas seguidas: 0                       (black: 0                   )
      Movimento do peão: normal                  (black: normal              )

   Jogadas feitas: 1
   Tabuleiro atual:
   _________
0 | . . . . | 
1 | . . T . | 
2 | . . . . | 
3 | . . . . | 
  |_________|
    0 1 2 3 
   Próximo jogador: BLACK
      Stock de peças:    ['t', 'b', 'p', 'c']    (white: ['B', 'P', 'C']     )
      Capturas seguidas: 0                       (white: 0                   )
      Movimento do peão: normal                  (white: normal              )

   Jogadas feitas: 2
   Tabuleiro atual:
   _________
0 | . c . . | 
1 | . . T . | 
2 | . . . . | 
3 | . . . . | 
  |_________|
    0 1 2 3 
   Próximo

In [5]:
def func_winner(state, player):
    winner = state.have_winner()
    if winner != None:
        return infinity if winner == player else -infinity

    almost_winner = state.n_in_row(3)
    if almost_winner == None:
        return 0
    elif almost_winner == player:
        return 20
    elif almost_winner == other_player(player):
        return -20
    if almost_winner == "BOTH":
        used_pieces = state.player_used_pieces(player)
        if len(used_pieces) == 3:
            available_pieces = [p for p in state.player_pieces(player) if p not in used_pieces]
            for piece in available_pieces:
                possible_moves = state.empty_cells("")
                for move in possible_moves:
                    # Cria uma cópia do estado e aplica o movimento
                    clone = state.next_state(move)
                    # Verifica se o movimento resulta em vitória para o jogador
                    if clone.have_winner() == player:
                        return 20
        other_no_win = True
        for piece in used_pieces:  # Todas as peças do jogador
            possible_moves = state.possible_moves(piece)
            for move in possible_moves:
                # Cria uma cópia do estado e aplica o movimento
                clone = state.next_state(move)
                # Verifica se o movimento resulta em vitória para o jogador
                if clone.have_winner() == player:
                    return 15

                # Verifica se o movimento bloqueia uma vitória do oponente
                if clone.have_winner() == other_player(player):
                    other_no_win = False

        if not other_no_win:
            return -15
    return 0



state = EstadoTicTacChess(to_move="test",board={'B': (0, 2), 'c': (3, 2), 'P': (1, 2), 't': (3, 3), 'T': (2, 2), 'p': (0, 3), 'b': (2, 0), 'C': (1,3)},n_jogadas=10,n_capturas=0,pawn_direction="1",last_piece="1")
#print(row_blocked(state, 3, "test"))
print(check_n_in_row(state.board, 3))

WHITE
